In [206]:
#!/usr/bin/env python
import os
from witb.utils import ioutils as io
from witb.globals import DATA_DIR
import time  
import nltk
import csv
import re, string
import collections
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import datetime

In [2]:
DATA_DIR="/home/sasha/Documents/MilaPostDoc/Python/CommonCrawl"

In [16]:
blacklistcsv='/home/sasha/Documents/MilaPostDoc/Python/porn_ngram_filter/wordlists/porn_blacklist.csv'

In [87]:
bigrams='/home/sasha/Documents/MilaPostDoc/Python/porn_ngram_filter/wordlists/porn_bigrams.csv'

In [147]:
bigramlist = []
with open(bigrams, newline='') as inputfile:
    for row in csv.reader(inputfile):
        bigramlist.append(tuple(row[0].split()))
print(len(bigramlist))

882


In [148]:
bigramlist

[('hairy', 'pussy'),
 ('wet', 'pussy'),
 ('black', 'pussy'),
 ('tight', 'pussy'),
 ('pussy', 'porn'),
 ('sex', 'videos'),
 ('pussy', 'sex'),
 ('shaved', 'pussy'),
 ('girl', 'pussy'),
 ('pussy', 'pics'),
 ('eating', 'pussy'),
 ('pussy', 'licking'),
 ('her', 'pussy'),
 ('pussy', 'free'),
 ('licking', 'pussy'),
 ('pink', 'pussy'),
 ('teen', 'pussy'),
 ('big', 'pussy'),
 ('pussy', 'fuck'),
 ('eat', 'pussy'),
 ('my', 'pussy'),
 ('naked', 'pussy'),
 ('fat', 'pussy'),
 ('pussy', 'and'),
 ('first', 'time'),
 ('girl', 'sex'),
 ('having', 'sex'),
 ('lesbian', 'pussy'),
 ('porn', 'movies'),
 ('fucking', 'pussy'),
 ('pussy', 'videos'),
 ('sex', 'pussy'),
 ('pussy', 'video'),
 ('skinny', 'teen'),
 ('girl', 'porn'),
 ('in', 'pussy'),
 ('porn', 'videos'),
 ('white', 'pussy'),
 ('free', 'porn'),
 ('sex', 'video'),
 ('and', 'pussy'),
 ('sex', 'with'),
 ('anal', 'sex'),
 ('sex', 'movies'),
 ('pussy', 'pictures'),
 ('little', 'pussy'),
 ('pussy', 'lips'),
 ('click', 'here'),
 ('to', 'watch'),
 ('girls', 

In [17]:
blacklist = []
with open(blacklistcsv, newline='') as inputfile:
    for row in csv.reader(inputfile):
        blacklist.append(row[0])
print(len(blacklist))

3811


In [3]:
filenames = os.listdir(DATA_DIR)
filename = os.path.join(DATA_DIR, filenames[0])

data = io.get_entries(io.read_wet_file(filename))
parsed_data = io.parse_data(data)  # Generator.

docs = []
start_time = time.time()
for doc in parsed_data:
    docs.append(doc)

In [160]:
test= str(docs[15].content)

In [167]:
#Simple word search
start = time.time()
pornsites={}
for doc in docs:
    pwds=len([w for w in word_tokenize(doc.content) if w in blacklist])
    pornsites[doc.url]=pwds
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

KeyboardInterrupt: 

In [ ]:
#Bigram search
stop_words = set(stopwords.words('english'))  
start = time.time()
pornsites={}
for doc in docs:
    clean = [w.strip(string.digits + string.punctuation).lower() for w in word_tokenize(test) if w not in stop_words]
    pwds=len([b for b in nltk.bigrams(word_tokenize(str(' '.join(clean)))) if b in bigramlist])
    pornsites[doc.url]=pwds
duration=str(datetime.timedelta(seconds=time.time() - start))
print("Number of pages counted: ", len(pornsites))
print("Duration: ", duration)

In [79]:
{k: v for k, v in sorted(pornsites.items(), key=lambda item: item[1], reverse=True)}

{'taylorholmes.com': 1819,
 'paularenee.wordpress.com': 1712,
 'lolcow.farm': 1458,
 'logs.test-lab.xenproject.org': 1427,
 'lyndamullalyhunt.wordpress.com': 854,
 'kalajaduvashikaranspecialist.com': 830,
 'zalbarath666.wordpress.com': 806,
 'chong-mail.net': 797,
 'www.pspbugs.com': 792,
 'verifiedspace.com': 776,
 'suimazate1989.no.comunidades.net': 771,
 'boto3.amazonaws.com': 766,
 'www.pornhavana.com': 764,
 'insecure.archiveofourown.org': 750,
 'adultinternetusers.com': 743,
 'www.allviewfunding.com': 661,
 'www.needlenthread.com': 652,
 'blog.gaymassagevideos.com': 634,
 'www.bikefriday.com': 592,
 'supportaustralia.wordpress.com': 534,
 'lovecraftzine.com': 528,
 'luxurylifeways.com': 502,
 'www.xn--80aafblbgpxxcgbigyfoeei.xn--p1ai': 500,
 'fanfaps.com': 500,
 'www.eurvehicle.com': 484,
 'www.maharaniweddings.com': 483,
 'www.dalgarnoinstitute.org.au': 459,
 'www.erieville.com': 449,
 'thetinypinkclover.wordpress.com': 419,
 'iammaurina.com': 412,
 'www.cloverfieldnews.com': 40

In [77]:
sorted_dict

OrderedDict([('0x0133c6f0becfbb55306c4549a162eb15e42ab69d.0xeth.com', 0),
             ('101things.com', 4),
             ('1024x.net', 4),
             ('156street-lausd-ca.schoolloop.com', 0),
             ('1920vidcofurnitoph.gazepti0fairnish.us.org', 8),
             ('1966mag.com', 27),
             ('199099.gov.sa:89', 12),
             ('1mt.top', 0),
             ('2012.igem.org', 0),
             ('2012.northernspark.org', 4),
             ('24hourcredit.info', 1),
             ('24ktgoldpoker.com', 5),
             ('32ndstreet-uscmagnet.schoolloop.com', 0),
             ('3dvideosystems.com', 10),
             ('3g.tbh16rg.cn', 2),
             ('3tfederal.com', 4),
             ('4b.wusdjqt.cn', 118),
             ('4g.haoyun205.cn', 47),
             ('5001.ws', 0),
             ('5amjoel.com', 14),
             ('987thecoast.com', 2),
             ('9fishgames.com', 2),
             ('aa.trinimuslims.com', 1),
             ('aaisp-btmso.blogspot.com', 0),
             ('a